In [14]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
wp_filetypes = 'https://en.wikipedia.org/wiki/List_of_file_formats'

In [9]:
page = requests.get(wp_filetypes)

In [13]:
soup = BeautifulSoup(page.text,'html.parser')

In [188]:
mydivs = soup.find_all("div", {"class": "mw-parser-output"})

In [198]:
children = [child for child in mydivs[0].children]